In [11]:
from collections import defaultdict
from glob import glob

import numpy as np
import pandas as pd
import re

NAN = float('nan')
STAGE_1_PATH = "data/1/fftoday*.csv"
STAGE_2_PATH = "data/2/{}.csv"

Throw away bad data, sort, rename stuff, etc. 

In [12]:
fftoday_files = glob(STAGE_1_PATH)
assert len(fftoday_files) > 0, "where are my files???"

In [13]:
# read CSVs from previous stage and consolidate by position
data = defaultdict(list)
for file in fftoday_files:
    df = pd.read_csv(file)
    data[re.sub(r'data/1/fftoday_([0-9]*)_pos', '', file.strip('.csv'))].append(df)

In [14]:
print(data.items())

dict_items([('30', [     Unnamed: 0                   name team  g  rec_target  rec_rec  rec_yd  \
0             1           Robert Woods  LAR  1          10        8     171   
1             2           Adam Thielen  MIN  1          12        8     166   
2             3         Devin Funchess  CAR  1           6        5      92   
3             4             A.J. Green  CIN  1           7        5     115   
4             5         Chester Rogers  IND  1           6        6     104   
5             6       Marquise Goodwin   SF  1           2        1      83   
6             7    JuJu Smith-Schuster  PIT  1           7        5      97   
7             8            Golden Tate  DET  1           7        6      97   
8             9         Brandon LaFell  CIN  1          10        6      95   
9            10          Davante Adams   GB  1           8        5      90   
10           11         Robby Anderson  NYJ  1           7        4      85   
11           12       Sterling S

In [17]:
for k, v in data.items():
    df = pd.concat(v)
    
    # get rid of 'Unnamed: 0' column
    df = df[df.columns[1:]]  
    
    # group chronologically by player
    df.sort_values(['name', 'season', 'week'], inplace=True)
    
    # remove players that only have 1 row, we can't use these for anything
    len_before = len(df)
    df = df[df.groupby('name').name.transform(len) > 1]
    len_lost = len_before - len(df)
    print(k, ":", len_before, "rows, {0:.0%}".format(len_lost/len_before), "unusable")
    
    # add column for previous week's points
    df['next_fp'] = np.where(df['name'] == df['name'].shift(-1), df['fp'].shift(-1), NAN)
    
    # Make sure number of columns is consistent (TE don't have rush data)
    if 'rush_att' not in df.columns:
        df['rush_att'] = NAN
    if 'rush_yd' not in df.columns:
        df['rush_yd'] = NAN
    if 'rush_td' not in df.columns:
        df['rush_td'] = NAN
        
    # save to CSV
    df.to_csv(STAGE_2_PATH.format(k))
    
    # print(df[['name', 'fp', 'next_fp']].head(20))

30 : 150 rows, 24% unusable
                name    fp  next_fp
3         A.J. Green  17.5     11.0
66        A.J. Green  11.0      6.6
136       A.J. Green   6.6      NaN
1       Adam Thielen  22.6     18.3
54      Adam Thielen  18.3      8.9
124     Adam Thielen   8.9      NaN
60    Alshon Jeffery  12.7     11.2
116   Alshon Jeffery  11.2      NaN
72      Amari Cooper   8.8      6.9
135     Amari Cooper   6.9      NaN
48     Antonio Brown   4.7     32.4
50     Antonio Brown  32.4     28.9
101    Antonio Brown  28.9      NaN
31     Brandin Cooks   7.4     20.9
53     Brandin Cooks  20.9     15.4
107    Brandin Cooks  15.4      NaN
8     Brandon LaFell  15.5      3.7
99    Brandon LaFell   3.7      NaN
23   Bruce Ellington  10.1      6.8
82   Bruce Ellington   6.8      NaN
20 : 150 rows, 19% unusable
                name    fp  next_fp
46   Adrian Peterson   4.2      3.9
90   Adrian Peterson   3.9      9.9
119  Adrian Peterson   9.9      NaN
61      Alex Collins  13.1     11.7
113     